In [1]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

/data/mutakabbir/Lightning/venv/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [15]:
DATABASE_TYPE="postgresql"
USERNAME="mutakabbir"
PASSWORD="lightning"
HOST="localhost"
PORT=5432
DATABASE_NAME="postgres"
FIRE_SUBDIVISION_TABLE = "fire_division_dataset"

In [3]:
engine = create_engine(f"{DATABASE_TYPE}://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}")

In [4]:
query_fire = """select f."YEAR", f."MONTH", f."DAY", f."REP_DATE", f."CALC_HA", f."CAUSE", f."geometry" from fire f"""
query_subdivision = """select sg."FEDUID", sg.geometry from subdivision_geo sg"""

In [5]:
fire_gdf = gpd.read_postgis(
    sql=query_fire, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326",)  

In [6]:
fire_gdf

,YEAR,MONTH,DAY,REP_DATE,CALC_HA,CAUSE,geometry
0,2006,4,9,2006-04-09,6.741532,H,"POLYGON ((-120.25300 55.87807, -120.25293 55.8..."
1,2006,4,15,2006-04-15,0.562148,H,"POLYGON ((-121.57551 55.60000, -121.57551 55.5..."
2,2006,5,29,2006-05-29,1.682437,H,"POLYGON ((-121.57539 55.46293, -121.57565 55.4..."
3,2006,5,18,2006-05-18,1.108936,H,"POLYGON ((-121.64146 55.70862, -121.64138 55.7..."
4,2006,4,24,2006-04-24,151.231090,H,"MULTIPOLYGON (((-120.47959 55.81054, -120.4791..."
...,...,...,...,...,...,...,...
59533,2006,7,17,2006-07-17,13.496292,L,"POLYGON ((-124.53606 55.46217, -124.53553 55.4..."
59534,2006,7,3,2006-07-03,181.530303,L,"MULTIPOLYGON (((-123.61535 55.55967, -123.6150..."
59535,2006,7,4,2006-07-04,2786.378984,L,"POLYGON ((-125.32666 56.08734, -125.32603 56.0..."
59536,2006,7,14,2006-07-14,5.241929,L,"POLYGON ((-125.27132 56.06404, -125.27152 56.0..."


In [7]:
division_gdf = gpd.read_postgis(
    sql=query_subdivision, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326",)  

In [8]:
division_gdf

,FEDUID,geometry
0,10001,"MULTIPOLYGON (((-53.44465 46.67495, -53.44474 ..."
1,10002,"MULTIPOLYGON (((-53.77539 48.71901, -53.77552 ..."
2,10003,"MULTIPOLYGON (((-55.51195 49.45222, -55.51213 ..."
3,10004,"MULTIPOLYGON (((-55.60642 53.43093, -55.60651 ..."
4,10005,"MULTIPOLYGON (((-58.32368 49.25135, -58.32372 ..."
...,...,...
333,59041,"MULTIPOLYGON (((-123.39095 48.42932, -123.3910..."
334,59042,"MULTIPOLYGON (((-123.29718 49.51761, -123.2972..."
335,60001,"MULTIPOLYGON (((-137.42686 68.99357, -137.4270..."
336,61001,"MULTIPOLYGON (((-111.73394 70.35465, -111.7339..."


In [9]:
fire_in_subdivision_gdf = gpd.sjoin(
    fire_gdf,
    division_gdf,
    how="inner",
    predicate="within").drop("index_right", axis=1)

In [23]:
fire_in_subdivision_gdf = fire_in_subdivision_gdf.rename(
    {
        "YEAR": "year",
        "MONTH": "month",
        "DAY": "day",
        "REP_DATE": "start_date",
        "CALC_HA": "area_burnt_ha",
        "CAUSE": "cause",
        "FEDUID": "division_id"
    },
    axis = 1
)

In [25]:
# drop duplicates
fire_in_subdivision_gdf.drop_duplicates(inplace=True)

In [24]:
fire_in_subdivision_gdf.to_postgis(name=FIRE_SUBDIVISION_TABLE, con=engine, if_exists='replace', index=False)

In [13]:
# # Operation too long
# fire_overlaps_subdivision_gdf = gpd.sjoin(
#     fire_not_in_sub_division,
#     division_gdf,
#     how="inner",
#     predicate="overlaps").drop("index_right", axis=1)

In [26]:
fire_in_subdivision_gdf

,year,month,day,start_date,area_burnt_ha,cause,geometry,division_id
0,2006,4,9,2006-04-09,6.741532,H,"POLYGON ((-120.25300 55.87807, -120.25293 55.8...",59024
1,2006,4,15,2006-04-15,0.562148,H,"POLYGON ((-121.57551 55.60000, -121.57551 55.5...",59024
2,2006,5,29,2006-05-29,1.682437,H,"POLYGON ((-121.57539 55.46293, -121.57565 55.4...",59024
3,2006,5,18,2006-05-18,1.108936,H,"POLYGON ((-121.64146 55.70862, -121.64138 55.7...",59024
4,2006,4,24,2006-04-24,151.231090,H,"MULTIPOLYGON (((-120.47959 55.81054, -120.4791...",59024
...,...,...,...,...,...,...,...,...
57448,1986,5,15,1986-05-15,1889.871771,H,"POLYGON ((-65.47934 46.19769, -65.48483 46.197...",13004
57516,1992,5,31,1992-05-31,917.448030,H,"POLYGON ((-65.50372 45.65718, -65.51496 45.659...",13004
57451,1986,5,15,1986-05-15,340.023966,H,"POLYGON ((-66.69949 45.59053, -66.70179 45.590...",13008
57509,1991,6,10,1991-06-10,165.471269,H,"POLYGON ((-66.56889 45.25688, -66.57125 45.258...",13008
